In [ ]:
#INSTALACIÓN DE LIBRERIAS
!pip install Torch==1.8.1
!pip install TorchVision

  Using cached torch-1.8.1-cp39-cp39-win_amd64.whl (190.5 MB)
  Attempting uninstall: Torch
    Found existing installation: torch 2.0.1
    Uninstalling torch-2.0.1:
      Successfully uninstalled torch-2.0.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.2 requires torch==2.0.1, but you have torch 1.8.1 which is incompatible.

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached torch-2.0.1-cp39-cp39-win_amd64.whl (172.4 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.8.1
    Uninstalling torch-1.8.1:
      Successfully uninstalled torch-1.8.1



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install opencv-python


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import shutil
import cv2

In [ ]:
# Ruta de la carpeta que contiene las imágenes
ruta_carpeta = r'D:\Master\TFM\IDC_regular_ps50_idx5\00Prueba2'

# Lista para almacenar los arrays de las imágenes
listado_arrays_imagenes = []
listado_arrays_etiquetas = []

# Recorrer las subcarpetas '0' y '1'
for subcarpeta in ['0', '1']:
    subcarpeta_ruta = ruta_carpeta +"\\" + subcarpeta

    # Para cada archivo en la subcarpeta
    for nombre_archivo in os.listdir(subcarpeta_ruta):
        # Comprobar si el archivo es una imagen (esto dependerá de las extensiones de archivo de tus imágenes)
        if nombre_archivo.endswith('.png'):
            # Construir la ruta completa del archivo
            ruta_archivo = os.path.join(subcarpeta_ruta, nombre_archivo)

            # Leer la imagen y convertirla a un array
            img = cv2.imread(ruta_archivo)
            # Añadir el array a la lista
            listado_arrays_imagenes.append(img)
            # Añadir la etiqueta correspondiente a la lista de etiquetas
            listado_arrays_etiquetas.append(int(subcarpeta))

In [ ]:
import numpy as np

# Convierte listado_arrays_etiquetas a numpy array
etiquetas = np.array(listado_arrays_etiquetas)

# Obtén los índices de las imágenes con etiqueta 0 y selecciona los primeros 5000
indices_0 = np.where(etiquetas == 0)[0][:500]

# Haz lo mismo para las imágenes con etiqueta 1
indices_1 = np.where(etiquetas == 1)[0][:500]

# Concatena los índices de las imágenes de ambas etiquetas
indices = np.concatenate([indices_0, indices_1])

# Utiliza los índices para seleccionar las imágenes y etiquetas correspondientes
listado_arrays_imagenes = [listado_arrays_imagenes[i] for i in indices]
listado_arrays_etiquetas = [listado_arrays_etiquetas[i] for i in indices]


In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Asumimos que la lista 'listado_arrays_imagenes' existe y contiene las matrices de imágenes

# Verificar la forma de las matrices
shapes = set(img.shape for img in listado_arrays_imagenes)
if len(shapes) > 1:
    # Realizar operación de ajuste en las matrices para que tengan la misma forma
    max_shape = max(shapes, key=lambda x: np.prod(x))
    listado_arrays_imagenes = [np.pad(img, [(0, max_shape[0]-img.shape[0]),
                                           (0, max_shape[1]-img.shape[1]),
                                           (0, max_shape[2]-img.shape[2])],
                                      mode='constant', constant_values=0)
                               for img in listado_arrays_imagenes]

# Convertir la lista de imágenes en una matriz 4D
imagenes = np.stack(listado_arrays_imagenes)

# Obtener las dimensiones de las imágenes
num_imagenes, altura, anchura, canales = imagenes.shape

# Reshape para que las imágenes sean 2D
imagenes_2d = imagenes.reshape(num_imagenes, -1)

# Crear una instancia del escalador MinMaxScaler
scaler = MinMaxScaler()

# Aplicar el escalado a las imágenes
imagenes_escaladas = scaler.fit_transform(imagenes_2d)

# Reshape de nuevo a las dimensiones originales de las imágenes
imagenes_escaladas = imagenes_escaladas.reshape(num_imagenes, altura, anchura, canales)


In [ ]:
# Contar las etiquetas 1
count_1 = listado_arrays_etiquetas.count(1)
print(f'Cantidad de etiquetas 1: {count_1}')

# Contar las etiquetas 0
count_0 = listado_arrays_etiquetas.count(0)
print(f'Cantidad de etiquetas 0: {count_0}')

Cantidad de etiquetas 1: 500
Cantidad de etiquetas 0: 500


In [ ]:
# Import necessary libraries
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid

# Convert lists to tensors
imagenes = torch.stack([torch.Tensor(i) for i in imagenes])
etiquetas = torch.tensor(listado_arrays_etiquetas)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(imagenes, etiquetas, test_size=0.2, random_state=42)

# Define custom dataset
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create train and test datasets
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

# Define the model architecture
class Model(nn.Module):
    def __init__(self, num_neurons):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3, num_neurons, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc = nn.Linear(num_neurons*24*24, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.reshape(-1, self.num_flat_features(x))
        x = self.sigmoid(self.fc(x))
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

# Define the function to create the model
def crear_modelo(num_neurons):
    print("CREAR MODELO", num_neurons)
    model = Model(num_neurons)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters())
    return model, criterion, optimizer

# Define the checkpoint directory
checkpoint_dir = 'D:\Master\TFM\Checkpoints'

# Create checkpoint directory if it doesn't exist
os.makedirs(checkpoint_dir, exist_ok=True)

def save_ckpt(model, optimizer, epoch, num_neurons, checkpoint_dir):
    save_dir = os.path.join(checkpoint_dir, "checkpoint_{:04d}_{}.pth".format(epoch, num_neurons))

    cp = {
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'epoch': epoch,
    }

    torch.save(cp, save_dir)
    print("Saved checkpoint into {}".format(save_dir))


def load_ckpt(checkpoint_dir, num_neurons):
    last_epoch = 0
    checkpoint_file = ''

    for file in os.listdir(checkpoint_dir):
        if file.endswith('{}.pth'.format(num_neurons)):
            epoch_number = int(file.split('_')[1])
            if epoch_number > last_epoch:
                last_epoch = epoch_number
                checkpoint_file = file

    if last_epoch == 0:
        return None, 0
    else:
        print("Loaded checkpoint", os.path.join(checkpoint_dir, checkpoint_file))
        checkpoint = torch.load(os.path.join(checkpoint_dir, checkpoint_file))
        return checkpoint, checkpoint['epoch']

# Define the grid of parameters
param_grid = {
    'num_neurons': [10, 20, 30]
}

# Perform grid search
best_accuracy = 0.0
best_params = None

for params in ParameterGrid(param_grid):
    print("Using {} neurons".format(params['num_neurons']))
    model, criterion, optimizer = crear_modelo(params['num_neurons'])
    checkpoint, start_epoch = load_ckpt(checkpoint_dir, params['num_neurons'])
    if checkpoint is not None:
        model.load_state_dict(checkpoint['model'])
        optimizer.load_state_dict(checkpoint['optimizer'])

    train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

    for epoch in range(start_epoch, start_epoch + 10):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            inputs = inputs.permute((0,3,1,2))
            outputs = model(inputs)
            loss = criterion(outputs, labels.float().unsqueeze(1))
            loss.backward()
            optimizer.step()

        save_ckpt(model, optimizer, epoch + 1, params['num_neurons'], checkpoint_dir) # Save the checkpoint

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs = inputs.permute((0,3,1,2))
                outputs = model(inputs)
                predicted = (outputs >= 0.5).squeeze().long()
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print("Epoch: {}, Accuracy: {}".format(epoch + 1, accuracy))
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = params

print("Best Accuracy:", best_accuracy)
print("Best Parameters:", best_params)

Using 10 neurons
CREAR MODELO 10
Loaded checkpoint C:\Users\alvaroldg\Desktop\TFM_Checkpoints\checkpoint_0010_10.pth
Saved checkpoint into C:\Users\alvaroldg\Desktop\TFM_Checkpoints\checkpoint_0011_10.pth
Epoch: 11, Accuracy: 0.52
Saved checkpoint into C:\Users\alvaroldg\Desktop\TFM_Checkpoints\checkpoint_0012_10.pth
Epoch: 12, Accuracy: 0.52
Saved checkpoint into C:\Users\alvaroldg\Desktop\TFM_Checkpoints\checkpoint_0013_10.pth
Epoch: 13, Accuracy: 0.52
Saved checkpoint into C:\Users\alvaroldg\Desktop\TFM_Checkpoints\checkpoint_0014_10.pth
Epoch: 14, Accuracy: 0.52
Saved checkpoint into C:\Users\alvaroldg\Desktop\TFM_Checkpoints\checkpoint_0015_10.pth
Epoch: 15, Accuracy: 0.52
Saved checkpoint into C:\Users\alvaroldg\Desktop\TFM_Checkpoints\checkpoint_0016_10.pth
Epoch: 16, Accuracy: 0.52
Saved checkpoint into C:\Users\alvaroldg\Desktop\TFM_Checkpoints\checkpoint_0017_10.pth
Epoch: 17, Accuracy: 0.52
Saved checkpoint into C:\Users\alvaroldg\Desktop\TFM_Checkpoints\checkpoint_0018_10.p

In [ ]:
# PREDICCIONES

import os
from PIL import Image
import torchvision.transforms as transforms

# Define la transformación que necesitas aplicar en las imágenes
# Para hacerla compatible con tu modelo
transform = transforms.Compose([
    transforms.Resize((50,50)),      # Redimensionar a 50x50
    transforms.ToTensor(),           # Convertir a tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalizar como se hizo durante el entrenamiento
])

# Define la ruta de tu directorio
directory = "C:\\Users\\alvaroldg\\Desktop\\IDC_regular_ps50_idx5\\12821\\1"

# Obtiene una lista de todos los archivos en el directorio
image_files = os.listdir(directory)

# Itera sobre cada archivo de imagen
for image_file in image_files:
    # Abre la imagen usando PIL
    image_pred = Image.open(os.path.join(directory, image_file))

    # Aplica las transformaciones a la imagen
    image_p_tensor = transform(image_pred)

    # Añade una dimensión extra en la posición 0
    image_p_tensor = image_p_tensor.unsqueeze(0)

    # Asegúrate de que el modelo está en modo de evaluación
    model.eval()

    # Realiza la predicción
    with torch.no_grad():
        output_p = model(image_p_tensor)
        prediction = (output_p >= 0.5).squeeze().long()

    # Imprime la predicción
    print(f"The prediction for image {image_file} is: {prediction.item()}")


The prediction for image 12821_idx5_x1001_y1551_class1.png is: 1
The prediction for image 12821_idx5_x1001_y1601_class1.png is: 1
The prediction for image 12821_idx5_x1001_y1651_class1.png is: 1
The prediction for image 12821_idx5_x1001_y1701_class1.png is: 1
The prediction for image 12821_idx5_x1001_y1751_class1.png is: 1
The prediction for image 12821_idx5_x1051_y1101_class1.png is: 1
The prediction for image 12821_idx5_x1051_y1151_class1.png is: 1
The prediction for image 12821_idx5_x1051_y1501_class1.png is: 1
The prediction for image 12821_idx5_x1051_y1551_class1.png is: 1
The prediction for image 12821_idx5_x1051_y1601_class1.png is: 1
The prediction for image 12821_idx5_x1051_y1651_class1.png is: 1
The prediction for image 12821_idx5_x1051_y1701_class1.png is: 1
The prediction for image 12821_idx5_x1051_y1751_class1.png is: 1
The prediction for image 12821_idx5_x1101_y1051_class1.png is: 1
The prediction for image 12821_idx5_x1101_y1101_class1.png is: 1
The prediction for image 

In [ ]:
# Import necessary libraries
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid

# Convert lists to tensors
imagenes = torch.stack([torch.Tensor(i) for i in imagenes])
etiquetas = torch.tensor(listado_arrays_etiquetas)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(imagenes, etiquetas, test_size=0.2, random_state=42)

# Define custom dataset
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create train and test datasets
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

# Define the model architecture
class Model(nn.Module):
    def __init__(self, num_neurons):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3, num_neurons, 3)
        self.conv2 = nn.Conv2d(num_neurons, num_neurons*2, 3)
        self.conv3 = nn.Conv2d(num_neurons*2, num_neurons*4, 3)
        self.conv4 = nn.Conv2d(num_neurons*4, num_neurons*8, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(num_neurons*8, num_neurons)  # Asume que después de cuatro convoluciones y max pooling, la dimensión es num_neurons * 8 * 3 * 3
        self.fc2 = nn.Linear(num_neurons, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

# Define the function to create the model
def crear_modelo(num_neurons):
    print("CREAR MODELO", num_neurons)
    model = Model(num_neurons)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters())
    return model, criterion, optimizer

# Define the checkpoint directory
checkpoint_dir = 'C:\\Users\\alvaroldg\\Desktop\\TFM_Checkpoints'

# Create checkpoint directory if it doesn't exist
os.makedirs(checkpoint_dir, exist_ok=True)

def save_ckpt(model, optimizer, epoch, num_neurons, checkpoint_dir):
    save_dir = os.path.join(checkpoint_dir, "checkpoint_{:04d}_{}.pth".format(epoch, num_neurons))

    cp = {
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'epoch': epoch,
    }

    torch.save(cp, save_dir)
    print("Saved checkpoint into {}".format(save_dir))


def load_ckpt(checkpoint_dir, num_neurons):
    last_epoch = 0
    checkpoint_file = ''

    for file in os.listdir(checkpoint_dir):
        if file.endswith('{}.pth'.format(num_neurons)):
            epoch_number = int(file.split('_')[1])
            if epoch_number > last_epoch:
                last_epoch = epoch_number
                checkpoint_file = file

    if last_epoch == 0:
        return None, 0
    else:
        print("Loaded checkpoint", os.path.join(checkpoint_dir, checkpoint_file))
        checkpoint = torch.load(os.path.join(checkpoint_dir, checkpoint_file))
        return checkpoint, checkpoint['epoch']

# Define the grid of parameters
param_grid = {
    'num_neurons': [10, 20, 30],
}

# Perform grid search
best_accuracy = 0.0
best_params = None

for params in ParameterGrid(param_grid):
    print("Using {} neurons".format(params['num_neurons']))
    model, criterion, optimizer = crear_modelo(params['num_neurons'])
    checkpoint, start_epoch = load_ckpt(checkpoint_dir, params['num_neurons'])
    if checkpoint is not None:
        model.load_state_dict(checkpoint['model'])
        optimizer.load_state_dict(checkpoint['optimizer'])

    train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

    for epoch in range(start_epoch, start_epoch + 10):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            inputs = inputs.permute((0,3,1,2))
            outputs = model(inputs)
            loss = criterion(outputs, labels.float().unsqueeze(1))
            loss.backward()
            optimizer.step()

        save_ckpt(model, optimizer, epoch + 1, params['num_neurons'], checkpoint_dir) # Save the checkpoint

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs = inputs.permute((0,3,1,2))
                outputs = model(inputs)
                predicted = (outputs >= 0.5).squeeze().long()
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print("Epoch: {}, Accuracy: {}".format(epoch + 1, accuracy))
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = params

print("Best Accuracy:", best_accuracy)
print("Best Parameters:", best_params)

Using 10 neurons
CREAR MODELO 10
Loaded checkpoint C:\Users\alvaroldg\Desktop\TFM_Checkpoints\checkpoint_0020_10.pth


RuntimeError: Error(s) in loading state_dict for Model:
	Missing key(s) in state_dict: "conv2.weight", "conv2.bias", "conv3.weight", "conv3.bias", "conv4.weight", "conv4.bias", "fc1.weight", "fc1.bias", "fc2.weight", "fc2.bias". 
	Unexpected key(s) in state_dict: "fc.weight", "fc.bias". 